In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from global_snowmelt_runoff_onset.config import Config
import random

In [ ]:
def validate_dataset_comparison(target_config_path, reference_config_path, n_tiles_to_check=10, tolerance=1e-6):
    """Compare two datasets for consistency"""
    from global_snowmelt_runoff_onset.config import Config
    import random
    import xarray as xr
    import numpy as np
    from scipy import stats
    
    # Load both configs

    print("target config:")
    config_v2 = Config(target_config_path)

    print("reference config:")
    config_v1 = Config(reference_config_path)


    # Get processed tiles from both versions
    tiles_v1 = config_v1.get_list_of_tiles(which='processed')
    tiles_v2 = config_v2.get_list_of_tiles(which='processed')
    
    # Find common tiles
    tiles_v1_indices = set((tile.row, tile.col) for tile in tiles_v1)
    tiles_v2_indices = set((tile.row, tile.col) for tile in tiles_v2)
    common_indices = tiles_v1_indices.intersection(tiles_v2_indices)
    
    if len(common_indices) == 0:
        return {"error": "No common processed tiles found"}
    
    # Sample tiles for detailed comparison
    sampled_indices = random.sample(list(common_indices), min(n_tiles_to_check, len(common_indices)))
    
    results = {
        'dataset_configs': {
            'target_dataset': target_config_path,
            'reference_dataset': reference_config_path,
            'target_processed_tiles': len(tiles_v2),
            'reference_processed_tiles': len(tiles_v1),
            'common_tiles': len(common_indices),
            'tiles_checked': len(sampled_indices)
        },
        'tile_comparisons': {}
    }
    
    try:
        # Open with mask_and_scale=False for detailed encoding comparison
        ds_v1_raw = xr.open_zarr(config_v1.global_runoff_store, consolidated=True, mask_and_scale=False)
        ds_v2_raw = xr.open_zarr(config_v2.global_runoff_store, consolidated=True, mask_and_scale=False)
        
        # Open with mask_and_scale=True for aggregation validation
        ds_v1_decoded = xr.open_zarr(config_v1.global_runoff_store, consolidated=True, mask_and_scale=True)
        ds_v2_decoded = xr.open_zarr(config_v2.global_runoff_store, consolidated=True, mask_and_scale=True)
        
        expected_ranges = {
            'runoff_onset': (1, 366),
            'runoff_onset_median': (1, 366),
            'runoff_onset_mad': (0, 100),
            'temporal_resolution': (0.1, 100),
            'temporal_resolution_median': (0.1, 100)
        }
        
        for row, col in sampled_indices:
            tile_v1 = config_v1.get_tile(row, col)
            tile_v2 = config_v2.get_tile(row, col)
            
            # Get tile bounds
            extent_v1 = tile_v1.geobox.extent.boundary.coords
            extent_v2 = tile_v2.geobox.extent.boundary.coords
            
            lons_v1, lats_v1 = zip(*extent_v1)
            min_lon_v1, max_lon_v1 = min(lons_v1), max(lons_v1)
            min_lat_v1, max_lat_v1 = min(lats_v1), max(lats_v1)
            
            lons_v2, lats_v2 = zip(*extent_v2)
            min_lon_v2, max_lon_v2 = min(lons_v2), max(lons_v2)
            min_lat_v2, max_lat_v2 = min(lats_v2), max(lats_v2)
            
            # Get raw data for encoding comparison
            tile_ds_v1_raw = ds_v1_raw.sel(
                latitude=slice(max_lat_v1, min_lat_v1),
                longitude=slice(min_lon_v1, max_lon_v1)
            ).compute()
            
            tile_ds_v2_raw = ds_v2_raw.sel(
                latitude=slice(max_lat_v2, min_lat_v2),
                longitude=slice(min_lon_v2, max_lon_v2)
            ).compute()
            
            # Get decoded data for aggregation validation
            tile_ds_v1_decoded = ds_v1_decoded.sel(
                latitude=slice(max_lat_v1, min_lat_v1),
                longitude=slice(min_lon_v1, max_lon_v1)
            ).compute()
            
            tile_ds_v2_decoded = ds_v2_decoded.sel(
                latitude=slice(max_lat_v2, min_lat_v2),
                longitude=slice(min_lon_v2, max_lon_v2)
            ).compute()
            
            tile_id = f"tile_{row}_{col}"
            tile_results = {'variables': {}}
            
            # Analyze each data variable in target dataset
            analysis_vars = [var for var in tile_ds_v2_raw.data_vars if var != 'spatial_ref']
            for var_name in analysis_vars:
                if var_name in tile_ds_v1_raw.data_vars:
                    var_comparison = analyze_variable_comparison(
                        tile_ds_v1_raw[var_name], tile_ds_v2_raw[var_name], var_name, expected_ranges, tolerance,
                        tile_ds_v1_decoded, tile_ds_v2_decoded
                    )
                    tile_results['variables'][var_name] = var_comparison
            
            results['tile_comparisons'][tile_id] = tile_results
        
    except Exception as e:
        results['error'] = f"Error during comparison: {str(e)}"
    
    return results


def analyze_variable_comparison(var_ref, var_target, var_name, expected_ranges, tolerance=1e-6, 
                              tile_ds_ref=None, tile_ds_target=None):
    """Analyze a single variable comparison between reference and target datasets"""
    import numpy as np
    from scipy import stats
    
    # Get encoding information for both datasets
    ref_encoding = var_ref.encoding
    ref_attrs = var_ref.attrs
    target_encoding = var_target.encoding 
    target_attrs = var_target.attrs
    
    # Raw values
    ref_raw = var_ref.values
    target_raw = var_target.values
    
    # Process reference dataset
    ref_fill = ref_encoding.get('_FillValue', ref_attrs.get('_FillValue'))
    ref_scale = ref_encoding.get('scale_factor', ref_attrs.get('scale_factor', 1.0))
    ref_offset = ref_encoding.get('add_offset', ref_attrs.get('add_offset', 0.0))
    
    ref_decoded = ref_raw.astype(np.float64)
    if ref_fill is not None:
        ref_valid_mask = ref_raw != ref_fill
        ref_decoded[~ref_valid_mask] = np.nan
    else:
        ref_valid_mask = np.ones(ref_raw.shape, dtype=bool)
    ref_decoded = ref_decoded * ref_scale + ref_offset
    ref_valid_values = ref_decoded[ref_valid_mask & ~np.isnan(ref_decoded)]
    
    # Process target dataset
    target_fill = target_encoding.get('_FillValue', target_attrs.get('_FillValue'))
    target_scale = target_encoding.get('scale_factor', target_attrs.get('scale_factor', 1.0))
    target_offset = target_encoding.get('add_offset', target_attrs.get('add_offset', 0.0))
    
    target_decoded = target_raw.astype(np.float64)
    if target_fill is not None:
        target_valid_mask = target_raw != target_fill
        target_decoded[~target_valid_mask] = np.nan
    else:
        target_valid_mask = np.ones(target_raw.shape, dtype=bool)
    target_decoded = target_decoded * target_scale + target_offset
    target_valid_values = target_decoded[target_valid_mask & ~np.isnan(target_decoded)]
    
    # Calculate statistics for both
    def get_stats(values, raw_values, valid_mask, decoded_values):
        if len(values) > 0:
            unique_vals = np.unique(values)
            try:
                mode_result = stats.mode(values, keepdims=False)
                mode_val = float(mode_result.mode)
            except:
                mode_val = None
                
            if len(unique_vals) > 1:
                diffs = np.diff(np.sort(unique_vals))
                min_diff = float(np.min(diffs[diffs > 0])) if np.any(diffs > 0) else 0.0
            else:
                min_diff = 0.0
                
            percentiles = {f'p{p:02d}': float(np.percentile(values, p)) for p in range(0, 101, 10)}
            
            return {
                'min': float(np.min(values)),
                'max': float(np.max(values)),
                'mean': float(np.mean(values)),
                'median': float(np.median(values)),
                'mode': mode_val,
                'std': float(np.std(values)),
                'unique_values': len(unique_vals),
                'precision': min_diff,
                'percentiles': percentiles
            }
        else:
            return {
                'min': None, 'max': None, 'mean': None, 'median': None,
                'mode': None, 'std': None, 'unique_values': 0, 'precision': None,
                'percentiles': {}
            }
    
    ref_stats = get_stats(ref_valid_values, ref_raw, ref_valid_mask, ref_decoded)
    target_stats = get_stats(target_valid_values, target_raw, target_valid_mask, target_decoded)
    
    # Calculate differences and matches
    num_mismatched = 0
    max_diff = 0.0
    min_diff = 0.0
    mean_diff = 0.0
    median_diff = 0.0
    mode_diff = None
    
    if len(ref_valid_values) > 0 and len(target_valid_values) > 0:
        # Only compare where both have valid data
        common_mask = ref_valid_mask & target_valid_mask
        if np.any(common_mask):
            ref_common = ref_decoded[common_mask]
            target_common = target_decoded[common_mask]
            diffs = np.abs(ref_common - target_common)
            max_diff = float(np.max(diffs))
            min_diff = float(np.min(diffs))
            mean_diff = float(np.mean(diffs))
            median_diff = float(np.median(diffs))
            num_mismatched = int(np.sum(diffs > tolerance))
            
            # Calculate mode of differences
            try:
                mode_result = stats.mode(diffs, keepdims=False)
                mode_diff = float(mode_result.mode)
            except:
                mode_diff = None
    
    # Check values outside expected range for both datasets
    target_values_outside_range = 0
    target_mode_outside_range = None
    ref_values_outside_range = 0
    ref_mode_outside_range = None
    
    if var_name in expected_ranges:
        min_exp, max_exp = expected_ranges[var_name]
        
        # Check target dataset
        if len(target_valid_values) > 0:
            outside_mask = (target_valid_values < min_exp) | (target_valid_values > max_exp)
            target_values_outside_range = int(np.sum(outside_mask))
            if target_values_outside_range > 0:
                outside_values = target_valid_values[outside_mask]
                try:
                    mode_result = stats.mode(outside_values, keepdims=False)
                    target_mode_outside_range = float(mode_result.mode)
                except:
                    target_mode_outside_range = None
        
        # Check reference dataset
        if len(ref_valid_values) > 0:
            outside_mask = (ref_valid_values < min_exp) | (ref_valid_values > max_exp)
            ref_values_outside_range = int(np.sum(outside_mask))
            if ref_values_outside_range > 0:
                outside_values = ref_valid_values[outside_mask]
                try:
                    mode_result = stats.mode(outside_values, keepdims=False)
                    ref_mode_outside_range = float(mode_result.mode)
                except:
                    ref_mode_outside_range = None
    
    # Build comparison table
    comparison = {
        'dimensions': {
            'target_dataset': str({dim: var_target.sizes[dim] for dim in var_target.dims}),
            'reference_dataset': str({dim: var_ref.sizes[dim] for dim in var_ref.dims}),
            'match': var_target.dims == var_ref.dims and all(var_target.sizes[dim] == var_ref.sizes[dim] for dim in var_target.dims if dim in var_ref.dims)
        },
        'chunk_dimensions': {
            'target_dataset': str(target_encoding.get('preferred_chunks', 'Not chunked')),
            'reference_dataset': str(ref_encoding.get('preferred_chunks', 'Not chunked')),
            'match': target_encoding.get('preferred_chunks') == ref_encoding.get('preferred_chunks')
        },
        'dtype': {
            'target_dataset': str(target_decoded.dtype),
            'reference_dataset': str(ref_decoded.dtype),
            'match': target_decoded.dtype == ref_decoded.dtype
        },
        'encoded_dtype': {
            'target_dataset': str(target_raw.dtype),
            'reference_dataset': str(ref_raw.dtype), 
            'match': target_raw.dtype == ref_raw.dtype
        },
        'nodata_value': {
            'target_dataset': target_fill,
            'reference_dataset': ref_fill,
            'match': target_fill == ref_fill
        },
        'encoded_nodata_value': {
            'target_dataset': target_fill,
            'reference_dataset': ref_fill,
            'match': target_fill == ref_fill
        },
        'scale_factor': {
            'target_dataset': target_scale,
            'reference_dataset': ref_scale,
            'match': abs(target_scale - ref_scale) < tolerance
        },
        'add_offset': {
            'target_dataset': target_offset,
            'reference_dataset': ref_offset,
            'match': abs(target_offset - ref_offset) < tolerance
        },
        'total_pixels': {
            'target_dataset': target_raw.size,
            'reference_dataset': ref_raw.size,
            'match': target_raw.size == ref_raw.size
        },
        'nan_pixels': {
            'target_dataset': int(np.count_nonzero(~target_valid_mask)),
            'reference_dataset': int(np.count_nonzero(~ref_valid_mask)),
            'match': np.count_nonzero(~target_valid_mask) == np.count_nonzero(~ref_valid_mask)
        },
        'unique_values': {
            'target_dataset': target_stats['unique_values'],
            'reference_dataset': ref_stats['unique_values'],
            'match': target_stats['unique_values'] == ref_stats['unique_values']
        },
        'precision': {
            'target_dataset': target_stats['precision'],
            'reference_dataset': ref_stats['precision'],
            'match': abs((target_stats['precision'] or 0) - (ref_stats['precision'] or 0)) < tolerance
        }
    }
    
    # Add statistical comparisons
    for stat in ['min', 'max', 'mean', 'median', 'mode', 'std']:
        target_val = target_stats[stat]
        ref_val = ref_stats[stat]
        if target_val is not None and ref_val is not None:
            match = abs(target_val - ref_val) < tolerance
        else:
            match = target_val == ref_val
        
        comparison[stat] = {
            'target_dataset': target_val,
            'reference_dataset': ref_val,
            'match': match
        }
    
    # Add percentiles
    for p in range(0, 101, 10):
        p_key = f'p{p:02d}'
        target_val = target_stats['percentiles'].get(p_key)
        ref_val = ref_stats['percentiles'].get(p_key)
        if target_val is not None and ref_val is not None:
            match = abs(target_val - ref_val) < tolerance
        else:
            match = target_val == ref_val
            
        comparison[f'percentile_{p:02d}'] = {
            'target_dataset': target_val,
            'reference_dataset': ref_val,
            'match': match
        }
    
    # Add expected range metrics
    comparison['num_outside_expected_range'] = {
        'target_dataset': target_values_outside_range,
        'reference_dataset': ref_values_outside_range, 
        'match': target_values_outside_range == 0 and ref_values_outside_range == 0
    }
    comparison['mode_outside_expected_range'] = {
        'target_dataset': target_mode_outside_range,
        'reference_dataset': ref_mode_outside_range,
        'match': target_mode_outside_range is None and ref_mode_outside_range is None
    }
    
    # For median and MAD variables, add correctness check for both datasets
    # Use the decoded datasets that were passed in
    if ('median' in var_name or 'mad' in var_name) and tile_ds_target is not None and tile_ds_ref is not None:
        # Check aggregation correctness using the decoded datasets
        target_aggregation_correct, target_aggregation_diff = check_aggregation_correctness(
            var_name, tile_ds_target[var_name], tile_ds_target, tolerance
        )
        ref_aggregation_correct, ref_aggregation_diff = check_aggregation_correctness(
            var_name, tile_ds_ref[var_name], tile_ds_ref, tolerance
        )
        comparison['aggregation_correct'] = {
            'target_dataset': target_aggregation_correct,
            'reference_dataset': ref_aggregation_correct,
            'match': target_aggregation_correct and ref_aggregation_correct
        }
        comparison['aggregation_difference'] = {
            'target_dataset': target_aggregation_diff,
            'reference_dataset': ref_aggregation_diff,
            'match': (target_aggregation_diff or 0) < tolerance and (ref_aggregation_diff or 0) < tolerance
        }
    
    # Add comparison metrics at the end (with visual separation)
    comparison['_separator'] = {
        'target_dataset': '--- COMPARISON METRICS ---',
        'reference_dataset': '--- COMPARISON METRICS ---',
        'match': True
    }
    comparison['num_values_mismatch'] = {
        'target_dataset': num_mismatched,
        'reference_dataset': '-',
        'match': num_mismatched == 0
    }
    comparison['min_difference'] = {
        'target_dataset': min_diff,
        'reference_dataset': '-',
        'match': min_diff < tolerance
    }
    comparison['max_difference'] = {
        'target_dataset': max_diff,
        'reference_dataset': '-',
        'match': max_diff < tolerance
    }
    comparison['mean_difference'] = {
        'target_dataset': mean_diff,
        'reference_dataset': '-',
        'match': mean_diff < tolerance
    }
    comparison['median_difference'] = {
        'target_dataset': median_diff,
        'reference_dataset': '-',
        'match': median_diff < tolerance
    }
    comparison['mode_difference'] = {
        'target_dataset': mode_diff,
        'reference_dataset': '-',
        'match': (mode_diff or 0) < tolerance
    }
    
    return comparison


def check_aggregation_correctness(var_name, stored_aggregation, source_dataset, tolerance=1e-6):
    """Check if stored aggregated values match what we calculate from source data
    
    Args:
        var_name: Name of the aggregated variable (e.g., 'runoff_onset_median')
        stored_aggregation: The stored aggregated xarray DataArray to validate (should be decoded)
        source_dataset: The xarray dataset containing the source multi-year data (should be decoded)
        tolerance: Tolerance for numerical differences
    """
    import numpy as np
    
    # Determine the base variable name for the aggregation
    if 'runoff_onset_median' in var_name:
        base_var = 'runoff_onset'
        agg_type = 'median'
    elif 'runoff_onset_mad' in var_name:
        base_var = 'runoff_onset'
        agg_type = 'mad'
    elif 'temporal_resolution_median' in var_name:
        base_var = 'temporal_resolution'
        agg_type = 'median'
    else:
        # For unknown aggregation types, can't validate
        return True, 0.0  # Assume correct if we can't validate
    
    # Check if the base variable exists in the dataset
    if base_var not in source_dataset.data_vars:
        # Base variable not available, can't validate
        return True, 0.0  # Assume correct if we can't validate
    
    try:
        # Get the source data (should have water_year dimension)
        source_data = source_dataset[base_var]
        
        # Check if water_year dimension exists
        if 'water_year' not in source_data.dims:
            # No water_year dimension, can't validate
            return True, 0.0  # Assume correct if we can't validate
        
        # Calculate the expected aggregation along water_year dimension using xarray
        # Data should already be decoded since we opened with mask_and_scale=True
        if agg_type == 'median':
            # Calculate median along water_year dimension
            expected_aggregation = source_data.median(dim='water_year', skipna=True)
        elif agg_type == 'mad':
            # Calculate MAD (median absolute deviation) along water_year dimension
            median_vals = source_data.median(dim='water_year', skipna=True)
            # MAD = median(|x - median(x)|)
            abs_deviations = np.abs(source_data - median_vals)
            expected_aggregation = abs_deviations.median(dim='water_year', skipna=True)
        else:
            return True, 0.0  # Unknown aggregation type
        
        # Now compare: abs(stored_aggregation - expected_aggregation) < tolerance
        # Work directly with xarray DataArrays (both should be decoded)
        differences = np.abs(stored_aggregation - expected_aggregation)
        
        # Get the maximum difference (convert to numpy for this operation)
        max_difference = float(differences.max().values)
        
        # Check if ALL differences are within tolerance
        # Only count valid (non-NaN) pixels
        valid_diffs = differences.where(~np.isnan(differences))
        num_mismatched = int((valid_diffs > tolerance).sum().values)
        
        # Consider correct if ALL valid values match within tolerance
        is_correct = num_mismatched == 0
        
        return is_correct, max_difference
        
    except Exception as e:
        # If anything fails, assume correct (can't validate)
        print(f"Warning: Could not validate aggregation for {var_name}: {e}")
        return True, 0.0


def print_validation_summary(results):
    """Print validation results in table format"""
    import pandas as pd
    
    if 'error' in results:
        print(f"❌ ERROR: {results['error']}")
        return
    
    if 'dataset_configs' in results:
        # Dataset comparison summary
        config_info = results['dataset_configs']
        print("="*80)
        print("🔍 DATASET COMPARISON VALIDATION")
        print("="*80)
        print(f"Target Dataset: {config_info['target_dataset']}")
        print(f"Reference Dataset: {config_info['reference_dataset']}")
        print(f"Target Processed Tiles: {config_info['target_processed_tiles']}")
        print(f"Reference Processed Tiles: {config_info['reference_processed_tiles']}")
        print(f"Common Tiles: {config_info['common_tiles']}")
        print(f"Tiles Checked: {config_info['tiles_checked']}")
        print()
        
        # Print tables for each tile
        for tile_id, tile_data in results['tile_comparisons'].items():
            print(f"📍 {tile_id.upper()}")
            print("="*80)
            
            for var_name, var_comparison in tile_data['variables'].items():
                print(f"\n📊 Variable: {var_name}")
                print("-" * 60)
                
                # Create table data
                table_data = []
                for metric, values in var_comparison.items():
                    if metric == '_separator':
                        # Add a visual separator
                        table_data.append({
                            'Metric': '',
                            'Target Dataset': '',
                            'Reference Dataset': '',
                            'Match?': ''
                        })
                        table_data.append({
                            'Metric': '--- COMPARISON METRICS ---',
                            'Target Dataset': '--- COMPARISON METRICS ---', 
                            'Reference Dataset': '--- COMPARISON METRICS ---',
                            'Match?': ''
                        })
                    else:
                        match_symbol = "✅" if values['match'] else "❌" if values['match'] is not None else "?"
                        table_data.append({
                            'Metric': metric,
                            'Target Dataset': str(values['target_dataset']) if values['target_dataset'] is not None else 'None',
                            'Reference Dataset': str(values['reference_dataset']) if values['reference_dataset'] is not None else 'None',
                            'Match?': match_symbol
                        })
                
                # Create and display DataFrame with wider columns
                df = pd.DataFrame(table_data)
                pd.set_option('display.max_colwidth', 50)
                pd.set_option('display.width', 120)
                print(df.to_string(index=False, max_colwidth=50, justify='left'))
                print()
    
    else:
        print("❌ ERROR: Unexpected results structure")

In [ ]:
def save_validation_results(results, config_path_target, config_path_reference, output_dir="tile_data/quality_check"):
    """Save validation results to JSON file with organized directory structure"""
    import json
    import os
    from datetime import datetime
    from pathlib import Path
    
    # Extract version info from config paths
    def extract_version(config_path):
        # Extract version from path like '../config/global_config_v7.txt'
        filename = os.path.basename(config_path)
        if 'global_config_v' in filename:
            version = filename.split('global_config_v')[1].split('.')[0]
            return f"v{version}"
        return "unknown"
    
    target_version = extract_version(config_path_target)
    reference_version = extract_version(config_path_reference)
    
    # Create output directory structure
    comparison_dir = f"quality_check_{reference_version}_{target_version}"
    full_output_dir = Path(output_dir) / comparison_dir
    full_output_dir.mkdir(parents=True, exist_ok=True)
    
    # Create filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"validation_results_{reference_version}_{target_version}_{timestamp}.json"
    output_path = full_output_dir / filename
    
    # Add metadata to results
    results_with_metadata = {
        "metadata": {
            "comparison_name": f"{reference_version} vs {target_version}",
            "reference_version": reference_version,
            "target_version": target_version,
            "reference_config_path": config_path_reference,
            "target_config_path": config_path_target,
            "timestamp": datetime.now().isoformat(),
            "output_path": str(output_path)
        },
        "validation_results": results
    }
    
    # Custom JSON encoder for handling numpy types
    class NumpyEncoder(json.JSONEncoder):
        def default(self, obj):
            import numpy as np
            if isinstance(obj, np.integer):
                return int(obj)
            elif isinstance(obj, np.floating):
                return float(obj)
            elif isinstance(obj, np.ndarray):
                return obj.tolist()
            elif isinstance(obj, (np.bool_, bool)):
                return bool(obj)
            return super().default(obj)
    
    # Save to JSON
    try:
        with open(output_path, 'w') as f:
            json.dump(results_with_metadata, f, indent=2, cls=NumpyEncoder)
        
        print(f"✅ Validation results saved to: {output_path}")
        print(f"📁 Directory: {full_output_dir}")
        return str(output_path)
        
    except Exception as e:
        print(f"❌ Error saving results: {e}")
        return None


def load_validation_results(json_path):
    """Load validation results from JSON file"""
    import json
    from pathlib import Path
    
    try:
        json_path = Path(json_path)
        if not json_path.exists():
            print(f"❌ File not found: {json_path}")
            return None
            
        with open(json_path, 'r') as f:
            data = json.load(f)
        
        print(f"✅ Loaded validation results from: {json_path}")
        
        # Print metadata if available
        if "metadata" in data:
            metadata = data["metadata"]
            print(f"📊 Comparison: {metadata.get('comparison_name', 'Unknown')}")
            print(f"📅 Timestamp: {metadata.get('timestamp', 'Unknown')}")
            print(f"🎯 Target: {metadata.get('target_version', 'Unknown')}")
            print(f"📋 Reference: {metadata.get('reference_version', 'Unknown')}")
        
        return data
        
    except Exception as e:
        print(f"❌ Error loading results: {e}")
        return None


def list_validation_results(output_dir="tile_data/quality_check"):
    """List all available validation result files"""
    import os
    from pathlib import Path
    import json
    
    output_path = Path(output_dir)
    if not output_path.exists():
        print(f"❌ Directory not found: {output_path}")
        return []
    
    # Find all JSON files
    json_files = list(output_path.rglob("validation_results_*.json"))
    
    if not json_files:
        print(f"❌ No validation result files found in {output_path}")
        return []
    
    print(f"📁 Found {len(json_files)} validation result files:")
    print("="*80)
    
    file_info = []
    for json_file in sorted(json_files):
        try:
            with open(json_file, 'r') as f:
                data = json.load(f)
            
            metadata = data.get("metadata", {})
            comparison_name = metadata.get("comparison_name", "Unknown")
            timestamp = metadata.get("timestamp", "Unknown")
            
            # Extract tiles checked from results
            validation_results = data.get("validation_results", {})
            tiles_checked = validation_results.get("dataset_configs", {}).get("tiles_checked", 0)
            
            relative_path = json_file.relative_to(output_path)
            
            print(f"📄 {relative_path}")
            print(f"   🔍 Comparison: {comparison_name}")
            print(f"   📅 Created: {timestamp}")
            print(f"   🎯 Tiles Checked: {tiles_checked}")
            print()
            
            file_info.append({
                "path": str(json_file),
                "relative_path": str(relative_path),
                "comparison_name": comparison_name,
                "timestamp": timestamp,
                "tiles_checked": tiles_checked
            })
            
        except Exception as e:
            print(f"❌ Error reading {json_file}: {e}")
    
    return file_info

In [ ]:
# Test dataset comparison
comparison_results = validate_dataset_comparison(
    reference_config_path='../config/global_config_v7.txt',
    target_config_path='../config/global_config_v8.txt',
    n_tiles_to_check=3,
    tolerance=1e-3  # Configurable tolerance for matching
)
print_validation_summary(comparison_results)

In [ ]:
# Save the validation results
saved_path = save_validation_results(
    comparison_results,
    config_path_target='../config/global_config_v8.txt',
    config_path_reference='../config/global_config_v7.txt',
    output_dir="tile_data/quality_check"  # Adjust path as needed
)

print(f"\n{'='*60}")
print("📋 USAGE EXAMPLES:")
print("="*60)

# Show how to list available files
print("\n🔍 To list all validation files:")
print("list_validation_results('tile_data/quality_check')")

if saved_path:
    print(f"\n📂 To load this specific result:")
    print(f"loaded_data = load_validation_results('{saved_path}')")
    print(f"validation_results = loaded_data['validation_results']")
    print(f"print_validation_summary(validation_results)")

In [ ]:
# Demonstrate listing available files
print("\n🔍 LISTING AVAILABLE VALIDATION FILES:")
print("="*60)
available_files = list_validation_results('tile_data/quality_check')

# Demonstrate loading the results back
if saved_path:
    print(f"\n📂 LOADING RESULTS FROM: {saved_path}")
    print("="*60)
    loaded_data = load_validation_results(saved_path)
    
    if loaded_data:
        print(f"\n🎯 Loaded data contains:")
        print(f"   - metadata: {list(loaded_data.get('metadata', {}).keys())}")
        print(f"   - validation_results: {list(loaded_data.get('validation_results', {}).keys())}")
        
        # You can now use the loaded data with print_validation_summary
        # print_validation_summary(loaded_data['validation_results'])
        print(f"\n✅ Ready to use with: print_validation_summary(loaded_data['validation_results'])")

In [ ]:
print_validation_summary(loaded_data['validation_results'])